In [1]:
import numpy as np
import pandas as pd
import mne
import sys
import platform
import os

if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell.
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')
from base.files_in_out import getListOfFiles,GetFiles

## Code to plot and save the figures of the cluster centers

In [5]:
cluster_epo= mne.read_epochs('ana/prestate/clus_center/wholegfp/cfa_n_vep_prestate_wholegfp_cluster_centers_5_epo.fif')
info_epo=mne.read_epochs('preproc/g01/g01_prestate/g01_n_tsk_cfa_vep_gfp_epo.fif').info

Reading /Volumes/BBC/BBC/WP1/data/EEG/tsk/ana/prestate/clus_center/wholegfp/cfa_n_vep_prestate_wholegfp_cluster_centers_5_epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =       0.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
5 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading /Volumes/BBC/BBC/WP1/data/EEG/tsk/preproc/g01/g01_prestate/g01_n_tsk_cfa_vep_gfp_epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =       0.00 ...       0.00 ms
        0 CTF compensation matrices available
Adding metadata with 40 columns
415 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [3]:
%matplotlib qt

import matplotlib.pyplot as plt
for n,topo in enumerate(cluster_epo.get_data()):
    map_data=topo.flatten()
    if n ==3:
        map_data=map_data*-1
    mne.viz.plot_topomap(map_data,info_epo)
    plt.savefig(f'ana/prestate/clus_center/wholegfp/whole_gfp_clus5_map{n}.svg',format='svg',dpi=900)

NameError: name 'cluster_epo' is not defined

In [3]:

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib qt

In [4]:
%load_ext autoreload

%autoreload 2

plt.style.use('default')
import matplotlib as mpl
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False


## Code to plo the figures of the gev and time coverage means
obtains box plots

In [5]:
n_clus=5
phy_phase='rsp_phase'
micro_type='wholegfp'
param_path='ana/prestate/param df/'
param_fn=f'{micro_type}/{phy_phase}_{n_clus}_clusters_tsk_n_prestate_{micro_type}.feather'
param_fp=param_path+param_fn

In [6]:
param_fn

'wholegfp/rsp_phase_5_clusters_tsk_n_prestate_wholegfp.feather'

In [7]:
df=pd.read_feather(param_fp)

In [8]:
df.columns

Index(['id', 'map', 'g_num', 'phy_cond', 'awareness', 'corr_mean_', 'gev_',
       'occurrences_', 'meandurs_', 'timecov_'],
      dtype='object')

In [12]:
df

,id,map,g_num,phy_cond,awareness,corr_mean_,gev_,occurrences_,meandurs_,timecov_
0,0,0,g01,inh,aware,0.601534,0.065071,34.765432,0.004616,0.160494
1,1,0,g01,inh,unaware,0.659848,0.097641,33.129412,0.006392,0.211765
2,2,0,g01,exh,aware,0.589717,0.083505,42.965035,0.005046,0.216783
3,3,0,g01,exh,unaware,0.583009,0.075591,41.056604,0.004825,0.198113
4,4,0,g03,inh,aware,0.641108,0.057804,32.768000,0.004150,0.136000
...,...,...,...,...,...,...,...,...,...,...
575,111,4,g51,exh,unaware,0.724348,0.091740,33.645714,0.005095,0.171429
576,112,4,g52,inh,aware,0.647575,0.076969,36.173913,0.004808,0.173913
577,113,4,g52,inh,unaware,0.624656,0.063132,36.821918,0.004464,0.164384
578,114,4,g52,exh,aware,0.650073,0.068501,40.778761,0.004340,0.176991


In [9]:
dv_columns=['timecov_','gev_']

In [10]:
figpath='ana/results_report/images/prestate/'

In [11]:
if phy_phase=='cardiac_phase':
    palette=dict(sys='tab:orange',dia='b')
else:
    palette=dict(inh='g',exh='tab:pink')

fill_styles=dict(aware='filled',unaware='empty')

In [26]:
for maps in df['map'].unique()[:1]:
    #select map to loop into
    df_map= df[df['map']==maps]
    for dv_col in dv_columns[:1]:
        #define df with awa and phy cond as columns
        df_awa=df_map.pivot_table(index=['g_num'],values=[dv_col],columns=['awareness','phy_cond'])
        df_awa=df_awa[dv_col]
        df_phy_cond=df_map.pivot_table(index=['g_num'],values=[dv_col],columns=['phy_cond','awareness'])
        df_phy_cond=df_phy_cond[dv_col]

        # plot section
        ## generate image and axes, ax1 = plot for awareness as X, ax2 plot for phy cond as X
        fig, [ax1,ax2] = plt.subplots(1,2)
        # place boxplots in pairs
        positions= [1,2,4,5]

        # draw first boxplot for awareness
        boxplot_1=ax1.boxplot(df_awa, positions=positions,patch_artist=True)
        # define palette based on colum label = sanity check to have that condition right
        colors_awa = [palette[k] for col in df_awa.columns for k, item in palette.items() if k in col]
        # simply double colors from palette to be able to color whiskers which are double
        double_color_awa=[item for item in colors_awa for _ in range(2)]
        #determines whether it should be filled (aware) or empty (unaware)
        fill_awa = [fill_styles[k] for col in df_awa.columns for k, item in fill_styles.items() if k in col]

        # draw black median
        for median in boxplot_1['medians']:
            median.set_color('black')
            median.set_linewidth(2)

        # color boxplot and outliers based on list generated before
        for patch,outlier,color, fill in zip(boxplot_1['boxes'], boxplot_1['fliers'],colors_awa, fill_awa):
            patch.set_edgecolor(color)
            patch.set_linewidth(2)
            outlier.set_markersize(5)
            outlier.set_markeredgecolor(color)
            if fill== 'filled':
                patch.set_facecolor(color)
            else:
                patch.set_facecolor('white')
        # color whiskers
        for whisker, cap, color in zip(boxplot_1['whiskers'],boxplot_1['caps'], double_color_awa):
            whisker.set_color(color)
            cap.set_color(color)

    #repeat exact same procedure for complementary boxplot (phy_cond as X axis)

        boxplot_2= ax2.boxplot(df_phy_cond,positions=positions ,patch_artist=True)
        colors_phy_cond = [palette[k] for col in df_phy_cond.columns for k, item in palette.items() if k in col]
        double_color_phy_cond=[item for item in colors_phy_cond for _ in range(2)]
        fill_phy_cond = [fill_styles[k] for col in df_phy_cond.columns for k, item in fill_styles.items() if k in col]

        for patch,outlier, color, fill in zip(boxplot_2['boxes'], boxplot_2['fliers'],colors_phy_cond, fill_phy_cond):
            patch.set_edgecolor(color)
            patch.set_linewidth(2)
            outlier.set_markersize(5)
            outlier.set_markeredgecolor(color)
            if fill== 'filled':
                patch.set_facecolor(color)
            else:
                patch.set_facecolor('white')

        for whisker, cap, color in zip(boxplot_2['whiskers'],boxplot_2['caps'], double_color_phy_cond):
            whisker.set_color(color)
            #whisker.set_linewidth(2)
            cap.set_color(color)



        for median in boxplot_2['medians']:
            median.set_color('black')
            median.set_linewidth(2)





In [ ]:
def plot_boxplot(ax, df, positions, palette, fill_styles, box_width=0.5, linewidth=2):
    """
    Function to generate and style boxplots with the given data.
    Arguments:
    - ax: the axis where the boxplot will be drawn
    - df: the dataframe for the boxplot
    - positions: positions for the boxes
    - palette: color palette for styling
    - fill_styles: fill styles for boxes
    - box_width: width of the boxes (default is 0.5)

    - linewidth: the width of the lines for all components (default is 2)
    """
    boxplot = ax.boxplot(df, positions=positions, patch_artist=True, widths=box_width)

    # Prepare the color palette and fill styles based on the columns
    colors = [palette[k] for col in df.columns for k, item in palette.items() if k in col]
    double_color = [item for item in colors for _ in range(2)]
    fills = [fill_styles[k] for col in df.columns for k, item in fill_styles.items() if k in col]


    # Style boxplot components (boxes, fliers, whiskers, etc.) with linewidth
    for patch, outlier, color, fill in zip(boxplot['boxes'], boxplot['fliers'], colors, fills):
        patch.set_edgecolor(color)
        patch.set_linewidth(linewidth)
        outlier.set_markersize(5)
        outlier.set_markeredgewidth(linewidth)
        outlier.set_markeredgecolor(color)
        outlier.set_markerfacecolor('white')  # Optional, change as per your preference
        patch.set_facecolor(color if fill == 'filled' else 'white')

    # Style whiskers and caps with linewidth
    for whisker, cap, color in zip(boxplot['whiskers'], boxplot['caps'], double_color):
        whisker.set_color(color)
        cap.set_color(color)
        whisker.set_linewidth(linewidth)
        cap.set_linewidth(linewidth)

    # Style the medians with linewidth
    for median in boxplot['medians']:
        median.set_color('black')
        median.set_linewidth(linewidth)


In [42]:

# Main loop to generate plots for maps and dv_columns
for maps in df['map'].unique()[:1]:
    # Select map to loop into
    df_map = df[df['map'] == maps]

    for dv_col in dv_columns[:1]:
        # Define df with awa and phy cond as columns
        df_awa = df_map.pivot_table(index=['g_num'], values=[dv_col], columns=['awareness', 'phy_cond'])[dv_col]
        df_phy_cond = df_map.pivot_table(index=['g_num'], values=[dv_col], columns=['phy_cond', 'awareness'])[dv_col]

        # Create the figure and axes
        fig, [ax1, ax2] = plt.subplots(1, 2)

        # Set positions for the boxplots
        positions = [1, 2, 4, 5]

        # Plot first boxplot for awareness
        plot_boxplot(ax1, df_awa, positions, palette, fill_styles, box_width=0.8,  linewidth=2)

        # Plot second boxplot for phy_cond
        plot_boxplot(ax2, df_phy_cond, positions, palette, fill_styles, box_width=0.8, linewidth=2)

In [50]:
%matplotlib qt

fig, [ax1,ax2] = plt.subplots(1,2)
ax1.boxplot(df_awa[dv_col], patch_artist=True, showmeans=True, meanline=True)
#ax1.boxplot((2,[df_map[df_map['awareness']=='unaware']['timecov_']]))

plt.show()

In [40]:
outlier.set_markeredgewidth(5)

In [38]:
df_map

,awareness,timecov_
0,aware,0.160494
1,unaware,0.211765
2,aware,0.216783
3,unaware,0.198113
4,aware,0.136000
...,...,...
111,unaware,0.188571
112,aware,0.195652
113,unaware,0.232877
114,aware,0.203540


In [115]:
%matplotlib qt
for dvs in dv_columns:
    for maps in df['map'].unique():
        df_map=df[df['map']==maps]
        print(f'results for dv {dvs}')
        print(f'Map n {maps}')
        print(pd.pivot_table(df_map,values=dvs,index=['awareness','phy_cond']))
        #print(pg.pairwise_tests(df_map,dv=dvs, subject='g_num',within=['phy_cond','awareness']))
        #print(pg.pairwise_tests(df_map,dv=dvs, subject='g_num',within=['awareness','phy_cond']))

        fig, [ax1,ax2] = plt.subplots(1,2)
        title= f'Map: {maps}'
        sns.boxplot(data=df_map,x='phy_cond',y=dvs, hue='awareness',fill=False,palette=palette, ax=ax1)
        sns.boxplot(data=df_map,x='awareness',y=dvs, hue='phy_cond', fill=False,palette=palette,ax=ax2)
        ax1.set_title(title)
        figname=f'map{maps}_{dvs}{cond[:3]}.svg'
        figpathname=figpath+figname
        plt.savefig(figpathname,dpi=1000,format='svg')



results for dv timecov_
Map n 0
                    timecov_
awareness phy_cond          
aware     exh       0.161625
          inh       0.163744
unaware   exh       0.180594
          inh       0.178093
                 Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              phy_cond  0.000001      1     28  0.000001  0.001835  0.966135   
1             awareness  0.008048      1     28  0.008048  4.938567  0.034517   
2  phy_cond * awareness  0.000155      1     28  0.000155  0.180830  0.673912   

   p-GG-corr       ng2  eps  
0   0.966135  0.000004  1.0  
1   0.034517  0.030757  1.0  
2   0.673912  0.000610  1.0  


/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)


results for dv timecov_
Map n 1
                    timecov_
awareness phy_cond          
aware     exh       0.184152
          inh       0.198474
unaware   exh       0.180420
          inh       0.188324
                 Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              phy_cond  0.003581      1     28  0.003581  2.794286  0.105742   
1             awareness  0.001397      1     28  0.001397  1.076397  0.308382   
2  phy_cond * awareness  0.000299      1     28  0.000299  0.167236  0.685692   

   p-GG-corr       ng2  eps  
0   0.105742  0.016681  1.0  
1   0.308382  0.006574  1.0  
2   0.685692  0.001413  1.0  


/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)


results for dv timecov_
Map n 2
                    timecov_
awareness phy_cond          
aware     exh       0.215785
          inh       0.204959
unaware   exh       0.218474
          inh       0.211091
                 Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              phy_cond  0.002404      1     28  0.002404  1.951856  0.173362   
1             awareness  0.000564      1     28  0.000564  0.331255  0.569518   
2  phy_cond * awareness  0.000086      1     28  0.000086  0.065695  0.799585   

   p-GG-corr       ng2  eps  
0   0.173362  0.005382  1.0  
1   0.569518  0.001268  1.0  
2   0.799585  0.000193  1.0  


...

/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)


results for dv gev_
Map n 3
                        gev_
awareness phy_cond          
aware     exh       0.216234
          inh       0.227628
unaware   exh       0.221490
          inh       0.209213
                 Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              phy_cond  0.000006      1     28  0.000006  0.003464  0.953482   
1             awareness  0.001255      1     28  0.001255  0.693020  0.412184   
2  phy_cond * awareness  0.004063      1     28  0.004063  2.695299  0.111831   

   p-GG-corr       ng2  eps  
0   0.953482  0.000009  1.0  
1   0.412184  0.001981  1.0  
2   0.111831  0.006383  1.0  


/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)


results for dv gev_
Map n 4
                        gev_
awareness phy_cond          
aware     exh       0.072931
          inh       0.070877
unaware   exh       0.073016
          inh       0.073049
                 Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              phy_cond  0.000030      1     28  0.000030  0.062222  0.804839   
1             awareness  0.000037      1     28  0.000037  0.093163  0.762451   
2  phy_cond * awareness  0.000032      1     28  0.000032  0.095779  0.759247   

   p-GG-corr       ng2  eps  
0   0.804839  0.000317  1.0  
1   0.762451  0.000395  1.0  
2   0.759247  0.000338  1.0  


/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Applications/anaconda3/envs/mne_1_3/lib/python3.9/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
